In [147]:
# !pip install hsclient
TRIALS = 10
targets = ["beta.hydroshare.org", "www.hydroshare.org"]
clients = {}

In [148]:
from hsclient import HydroShare
for target in targets:
    hs = HydroShare(host=target, port=443, protocol='https')
    hs.sign_in()
    hs.resources = []
    clients[target] = hs

In [180]:
from time import time
from functools import wraps

diffs = []
  
def timer_func(func): 
    @wraps(func)
    def wrap_func(*args, **kwargs): 
        t1 = time() 
        result = func(*args, **kwargs) 
        t2 = time()
        diff = t2-t1
        diffs.append(diff)
        print(f'Function {func.__name__!r} executed in {(diff):.4f}s') 
        return result 
    return wrap_func


def create_res(hs, time=True):

    @timer_func
    def create_res_time(hs):
        resource = hs.create()
        hs.resources.append(resource)
    
    if time:
        create_res_time(hs)
    else:
        resource = hs.create()
        hs.resources.append(resource)

@timer_func
def delete_resources(hs):
    for res in hs.resources:
        res.delete()

@timer_func
def download_all_resources(hs):
    for res in hs.resources:
        res.download()

def download_single_resource(hs):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload('20.zip')

    @timer_func
    def download_single_resource_inner(res):
        res.download()

    download_single_resource_inner(res)

def upload_file(hs, res_number=-1):
    create_res(hs, False)
    res = hs.resources[-1]

    @timer_func
    def upload_file_inner(res):
        res.file_upload('20.zip')

    upload_file_inner(res)


def create_dir_upload_file(hs):
    create_res(hs, False)
    res = hs.resources[-1]

    @timer_func
    def create_dir_upload_file_inner(res):
        res.folder_create('New_Folder')
        res.file_upload('20.zip', destination_path='New_Folder')
    create_dir_upload_file_inner(res)

def upload_and_movefile(hs):
    create_res(hs, False)
    res = hs.resources[-1]
    
    @timer_func
    def upload_and_movefile_inner(res):
        res.folder_create('New_Folder')
        res.file_upload('20.zip', destination_path='New_Folder')
        file = res.file(path="New_Folder/20.zip")
        res.file_rename(file, '20.zip')
    upload_and_movefile_inner(res)

def download_single_file(hs):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload('20.zip')

    @timer_func
    def download_single_file_inner(res):
        file = res.file(path="20.zip")
        res.file_download(file)
    download_single_file_inner(res)

def delete_single_file(hs):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload('20.zip')

    @timer_func
    def delete_single_file_inner(res):
        file = res.file(path="20.zip")
        res.file_delete(file)
    delete_single_file_inner(res)
        

def run_comparisons(functions_to_run, runs=1):
    for funct in functions_to_run:
        print(f"********* Start {funct.__name__!r} *********")
        for target in targets:
            client = clients[target]
            diffs.clear()
            for i in range(runs):
                print(f"Starting run {i} of {funct.__name__} on {target}...")
                funct(client)
                print(diffs)
            print(f"Average time {funct.__name__!r} on {target}: {sum(diffs)/len(diffs)}\n\n")

def clear_resources():
    for target in targets:
        client = clients[target]
        client.resources = []
  

In [181]:
clear_resources()
run_comparisons([download_single_resource, upload_file, create_dir_upload_file, upload_and_movefile, delete_single_file, download_single_file, create_res ], runs=TRIALS)


********* Start 'download_single_resource' *********
Starting run 0 of download_single_resource on beta.hydroshare.org...
Function 'download_single_resource_inner' executed in 31.8460s
[31.846033811569214]
Starting run 1 of download_single_resource on beta.hydroshare.org...
Function 'download_single_resource_inner' executed in 37.1286s
[31.846033811569214, 37.12863779067993]
Starting run 2 of download_single_resource on beta.hydroshare.org...


In [ ]:
run_comparisons([download_all_resources, delete_resources], runs=1)